Authors:
Rajeswari Nakka, G.V.S.N.R.V.Prasad, R.Kiran Kumar

## Importing Libraries

In [6]:
import pandas as pd
import numpy as np
import math
import heapq
import joblib
from sklearn.model_selection import train_test_split

## Loading Data

In [7]:
df = joblib.load('/content/drive/MyDrive/CF Project/Datasets/Final 3 Datasets (Full Length)/finefoods.pkl')
df = df.rename(columns={'rating': 'Rating'})

d = (dict(enumerate(df['userId'].unique())))
new_d = {}
new_d = {value:key for key, value in d.items()}
df['userId'] = df['userId'].map(new_d)

d = (dict(enumerate(df['itemId'].unique())))
new_d = {}
new_d = {value:key for key, value in d.items()}
df['itemId'] = df['itemId'].map(new_d)

train, test = train_test_split(df[(df['userId']<1000) & (df['itemId']<500)], test_size=0.2)

In [8]:
train_dataframe = train
train_dataframe

,itemId,userId,Rating,summary,text
678,145,656,4.0,Very different flavor,I love these chips. The flavor is not plain. T...
514,135,503,5.0,"Delicious, what else did you expect?","Seriously, now. Tasty, tasty, tasty. Fresh, to..."
659,143,638,4.0,"Good, but container could be better",I used to buy the Wholesome Sweetners agave an...
901,192,872,3.0,Repeated deliveries of broken jars. Great pro...,I liked getting this one for my twin 10-month ...
249,74,247,5.0,keeps you out of the dentest chair,If used on a daily basis you will keep your gu...
...,...,...,...,...,...
860,187,834,3.0,Chocolate was all melt,The chocolate on the stick was all melted and ...
147,38,146,5.0,Love the salsa!!,This is one of the best salsas that I have fou...
68,19,67,3.0,How much would you pay for a bag of chocolate ...,"If you're impulsive like me, then $6 is ok. Do..."
217,61,216,3.0,More Hot/Spicy than McCormick's Brand,My husband and I are both kind of wimpy when i...


In [9]:
test_dataframe = test
test_dataframe

,itemId,userId,Rating,summary,text
602,135,460,5.0,"One bite and you'll become a ""chippoisseur""","As an Oregonian, I make a point of sharing the..."
855,186,829,2.0,Funny taste,"I can't eat these oats, they have a funny tast..."
888,188,861,5.0,Very Berry Snack Bars,"Recently diagnosed a gluten-intolerant, I have..."
472,135,465,4.0,Yummy chips,These are delicious. The 2 oz bags are a bit ...
625,135,605,5.0,"CRISPY, CRUNCHY, AND ROBUST","We've long enjoyed this brand of chip, so when..."
...,...,...,...,...,...
315,91,313,5.0,Excellent coffee,This brand of coffee has been available in my ...
799,174,774,5.0,Great item,This chocolate treat is a great gift and it ar...
15,9,15,5.0,"Lots of twizzlers, just what you expect.",My daughter loves twizzlers and this shipment ...
4,4,4,5.0,Great taffy,Great taffy at a great price. There was a wid...


TODO: Split the data in test and train

In [10]:
'''
CREATE PIVOT TABLE
'''
pivot_table = train_dataframe.pivot_table(values='Rating',columns='itemId',index='userId')
# REPLACE NULL VALUES WITH ZEROES
pivot_table_nn = pivot_table.fillna(0)
pivot_table_nn

itemId,0,2,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,40,41,42,43,...,199,200,201,202,203,204,205,206,207,208,210,211,212,213,214,215,216,237,266,287,318,339,342,345,346,356,359,366,382,388,407,438,439,440,459,465,471,497,498,499
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Algorithm** Adaptive Similarity Measure Model
</br>
Input: user-item rating matrix</br>
Output: user-movie rating matrix with predicted ratings</br>

In [11]:
''' 
FOR ALL ITEMS IN THE DATASET M
COMPUTE THE INVERSE USER FREQUENCY(IUF) FOR EACH ITEM m in M
'''
iuf_dict = {} # will store the item ids as keys and values as iuf value caculated
for item in pivot_table_nn.columns:
  item_ratings = pivot_table_nn[item]
  filtered_item_ratings = [x for x in item_ratings if x > 0.0]
  Um = len(filtered_item_ratings)
  U = len(item_ratings)
  iuf = math.log(U/Um)
  iuf_dict[item] = iuf

SHould i drop 0s when finidng similarities in all of the similarity functions

In [12]:
'''
FUCTION TO CALCULATE SIMILARITY_ASM
'''
def sim_asm(user_id1, user_id2):
  avg_u1_rating = np.array(pivot_table_nn.loc[user_id1]).mean()
  avg_u2_rating = np.array(pivot_table_nn.loc[user_id2]).mean()
  numerator = 0.0
  denominator = 0.0
  for item in pivot_table_nn.columns:
    iuf = iuf_dict[item]
    num_subterm1 = pivot_table_nn[item][user_id1] - avg_u1_rating
    num_subterm2 = pivot_table_nn[item][user_id2] - avg_u2_rating
    numerator += ((iuf**2) * num_subterm1 * num_subterm2)
  denominator = abs(numerator)**0.5
  print('numerator,denominator',numerator,denominator)
  similarity = numerator/denominator
  return similarity


call above and below function for each pair of user

In [13]:
'''
FUNCTION TO CALCULATE JACCARD SIMILARITY
'''
def sim_jaccard(user_id1, user_id2):
  numerator=0
  denominator=0
  for item in pivot_table_nn.columns:
    if pivot_table_nn[item][user_id1]>0 and pivot_table_nn[item][user_id2]>0:
      #if ratings for both users available for given movie increase numerator by 1
      numerator += 1 
    if pivot_table_nn[item][user_id1]>0 or pivot_table_nn[item][user_id2]>0:
      #if ratings for both users available for given movie increase numerator by 1
      denominator += 1 
  similarity = 1 - (numerator/denominator)
  return similarity

In [14]:
'''
FUNCTION TO CALCULATE MODIFIED SIMILARITY
'''
def modified_sim(user_id1, user_id2):
  term1 = sim_asm(user_id1, user_id2)
  term2 = sim_jaccard(user_id1, user_id2)
  print(term1,term2)
  modified_simialrity = term1 * term2
  print(modified_simialrity)
  return modified_simialrity

In [ ]:

#DICTIONARY
i=0
dict_users = {}
for user in pivot_table_nn.index:
  dict_users[user] = i
  i+=1
i=0

In [15]:
'''
CREATE A SIMILARITY MATRIX OF DIMENSION (NO. OF USERS *  NO. OF USERS)
'''

similarity_matrix = np.zeros([pivot_table_nn.shape[0],pivot_table_nn.shape[0]])
for user1 in pivot_table_nn.index:
  for user2 in pivot_table_nn.index:
    i+=1
    print("Cell Number:",i)
    if  similarity_matrix[dict_users[user1]][dict_users[user2]] == 0:
      similarity_matrix[dict_users[user1]][dict_users[user2]] = modified_sim(user1,user2)
      similarity_matrix[dict_users[user2]][dict_users[user1]] = similarity_matrix[dict_users[user1]][dict_users[user2]] 
similarity_matrix

Streaming output truncated to the last 5000 lines.
Cell Number: 646574
Cell Number: 646575
Cell Number: 646576
Cell Number: 646577
Cell Number: 646578
Cell Number: 646579
Cell Number: 646580
Cell Number: 646581
Cell Number: 646582
Cell Number: 646583
Cell Number: 646584
Cell Number: 646585
Cell Number: 646586
Cell Number: 646587
Cell Number: 646588
Cell Number: 646589
Cell Number: 646590
Cell Number: 646591
Cell Number: 646592
Cell Number: 646593
Cell Number: 646594
Cell Number: 646595
Cell Number: 646596
Cell Number: 646597
Cell Number: 646598
Cell Number: 646599
Cell Number: 646600
Cell Number: 646601
Cell Number: 646602
Cell Number: 646603
Cell Number: 646604
Cell Number: 646605
Cell Number: 646606
Cell Number: 646607
Cell Number: 646608
Cell Number: 646609
Cell Number: 646610
Cell Number: 646611
Cell Number: 646612
Cell Number: 646613
Cell Number: 646614
Cell Number: 646615
Cell Number: 646616
Cell Number: 646617
Cell Number: 646618
Cell Number: 646619
Cell Number: 646620
Cell Numb

array([[ 0.        , -2.20258842, -1.90075859, ..., -1.60782472,
        -1.60782472, -1.60782472],
       [-2.20258842,  0.        , -1.70009017, ..., -1.43808215,
        -1.43808215, -1.43808215],
       [-1.90075859, -1.70009017,  0.        , ..., -0.91080611,
        -0.91080611, -0.91080611],
       ...,
       [-1.60782472, -1.43808215, -0.91080611, ...,  0.        ,
         0.        ,  0.        ],
       [-1.60782472, -1.43808215, -0.91080611, ...,  0.        ,
         0.        ,  0.        ],
       [-1.60782472, -1.43808215, -0.91080611, ...,  0.        ,
         0.        ,  0.        ]])

In [16]:
#CREATE DATAFRAME
similarity_matrix_df = pd.DataFrame(similarity_matrix)
# similarity_matrix_df['original_user_id'] = list(dict_users.keys())
similarity_matrix_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806
0,0.000000,-2.202588,-1.900759,-2.125113,-2.125113,-2.462569,-2.462569,-2.462569,-1.101294,-1.499746,-3.101222,-1.060481,-1.676767,-1.676767,-1.676767,-1.676767,-1.676767,-1.676767,-1.676767,-1.676767,-1.676767,-0.749873,-1.499746,-2.462569,-2.462569,-1.482684,-1.482684,-1.657691,-1.482684,-1.657691,-1.657691,-1.482684,-1.657691,-1.657691,-1.284042,-2.968532,-1.284042,-1.482684,-1.284042,-0.741342,...,-1.801329,-1.801329,-1.801329,-1.801329,-1.801329,-1.801329,-1.801329,-0.805579,-1.139261,-2.462569,-1.972446,-1.527850,-1.972446,-1.972446,-1.972446,-2.462569,-1.907498,-1.557465,-2.462569,-2.462569,-1.101294,-2.462569,-1.607825,-1.607825,-1.016878,-1.016878,-1.607825,-1.607825,-1.607825,-1.607825,-1.607825,-1.607825,-1.607825,-1.607825,-1.607825,-1.607825,-1.607825,-1.607825,-1.607825,-1.607825
1,-2.202588,0.000000,-1.700090,-1.900759,-1.900759,-2.202588,-2.202588,-2.202588,-0.985027,-1.341414,-2.773817,-0.948523,-1.499746,-1.499746,-1.499746,-1.499746,-1.499746,-1.499746,-1.499746,-1.499746,-1.499746,-0.670707,-1.341414,-2.202588,-2.202588,-1.326153,-1.326153,-1.482684,-1.326153,-1.482684,-1.482684,-1.326153,-1.482684,-1.482684,-1.148482,-2.655136,-1.148482,-1.326153,-1.148482,-0.663076,...,-1.611158,-1.611158,-1.611158,-1.611158,-1.611158,-1.611158,-1.611158,-0.720532,-1.018986,-2.202588,-1.764209,-1.366550,-1.764209,-1.764209,-1.764209,-2.202588,-1.706118,-1.393039,-2.202588,-2.202588,-0.985027,-2.202588,-1.438082,-1.438082,-0.909523,-0.909523,-1.438082,-1.438082,-1.438082,-1.438082,-1.438082,-1.438082,-1.438082,-1.438082,-1.438082,-1.438082,-1.438082,-1.438082,-1.438082,-1.438082
2,-1.900759,-1.700090,0.000000,0.000000,0.000000,-1.900759,-1.900759,-1.900759,-0.850045,-0.899212,-2.115167,-0.635839,-1.005349,-1.005349,-1.005349,-1.005349,-1.005349,-1.005349,-1.005349,-1.005349,-1.005349,-0.449606,-0.899212,-1.900759,-1.900759,-0.876283,-0.876283,-0.979714,-0.876283,-0.979714,-0.979714,-0.876283,-0.979714,-0.979714,-0.758883,-2.138393,-0.758883,-0.876283,-0.758883,-0.438141,...,-1.165039,-1.165039,-1.165039,-1.165039,-1.165039,-1.165039,-1.165039,-0.521021,-0.736836,-1.900759,-1.368912,-1.060355,-1.368912,-1.368912,-1.368912,-1.900759,-1.472321,-1.202145,-1.900759,-1.900759,-0.850045,-1.900759,-0.910806,-0.910806,-0.576044,-0.576044,-0.910806,-0.910806,-0.910806,-0.910806,-0.910806,-0.910806,-0.910806,-0.910806,-0.910806,-0.910806,-0.910806,-0.910806,-0.910806,-0.910806
3,-2.125113,-1.900759,0.000000,0.000000,0.000000,-2.125113,-2.125113,-2.125113,-0.950379,-1.005349,-2.364828,-0.710889,-1.124014,-1.124014,-1.124014,-1.124014,-1.124014,-1.124014,-1.124014,-1.124014,-1.124014,-0.502675,-1.005349,-2.125113,-2.125113,-0.979714,-0.979714,-1.095354,-0.979714,-1.095354,-1.095354,-0.979714,-1.095354,-1.095354,-0.848457,-2.390796,-0.848457,-0.979714,-0.848457,-0.489857,...,-1.302554,-1.302554,-1.302554,-1.302554,-1.302554,-1.302554,-1.302554,-0.582520,-0.823807,-2.125113,-1.530491,-1.185513,-1.530491,-1.530491,-1.530491,-2.125113,-1.646105,-1.344039,-2.125113,-2.125113,-0.950379,-2.125113,-1.018312,-1.018312,-0.644037,-0.644037,-1.018312,-1.018312,-1.018312,-1.018312,-1.018312,-1.018312,-1.018312,-1.018312,-1.018312,-1.018312,-1.018312,-1.018312,-1.018312,-1.018312
4,-2.125113,-1.900759,0.000000,0.000000,0.000000,-2.125113,-2.125113,-2.125113,-0.950379,-1.005349,-2.364828,-0.710889,-1.124014,-1.124014,-1.124014,-1.124014,-1.124014,-1.124014,-1.124014,-1.124014,-1.124014,-0.502675,-1.005349,-2.125113,-2.125113,-0.979714,-0.979714,-1.095354,-0.979714,-1.095354,-1.095354,-0.979714,-1.095354,-1.095354,-0.848457,-2.390796,-0.848457,-0.979714,-0.848457,-0.489857,...,-1.302554,-1.302554,-1.302554,-1.302554,-1.302554,-1.302554,-1.302554,-0.582520,-0.823807,-2.125113,-1.530

In [17]:
import joblib
file_path = '/content/drive/MyDrive/CF Project/Datasets/similarity_matrix_df_finefood.sav'
joblib.dump(similarity_matrix_df,file_object)

UnsupportedOperation: ignored

In [ ]:
def find_delu(mu, user):
  # FIND DEL_U
  count_items_rated_by_user = len([x for x in pivot_table_nn.loc[user] if x > 0])
  term1 = 1 / count_items_rated_by_user
  summation = 0
  for r in pivot_table_nn.loc[user]:
    if r > 0:
      summation += (r-mu)
  del_u = term1 * summation
  return del_u
def find_deli(mu, user, del_u):
  # FIND DEL_I
  count_users_rated_item = len([x for x in pivot_table_nn[item]])
  term1 = 1 / count_users_rated_item
  summation = 0
  for r in pivot_table_nn[item]:
    if r > 0:
      summation += (r - del_u - mu)
  del_i = summation
  return del_i
def find_delui(mu,user):
  # FIND DEL_UI
  del_u = find_delu(mu,user)
  del_i = find_deli(mu,user,del_u)
  del_ui = mu + del_u + del_i
  return del_ui
def find_key(val, my_dict):
  for key, value in my_dict.items():
    if val == value:
      return key
  return "key doesn't exist"

In [ ]:
'''
PREDICT THE RATINGS
'''
# FIND MU, GLOBAL AVERAGE
mu = train_dataframe['Rating'].mean()
N = 5 # NUMBER OF NEIGHBOURS TO TAKE
mae = []
pred = []
actuals = []
for index,row in test_dataframe.iterrows():
  user = row['userId']
  item = row['itemId']
  given_rating = row['Rating']
  #GET NEIGHBOUR 'N' USERS
  if user not in dict_users.keys() or item not in pivot_table_nn.columns or dict_users[user] not in similarity_matrix_df.index:
    #cold start problem
    continue
  the_row = similarity_matrix_df[dict_users[user]]
  the_row = the_row.sort_values(ascending=False)
  # print('the_row',the_row[:N])
  neighbour_index = the_row.index
  neighbour_similarities = list(the_row)
  
  # FIND PRED_UI
  num = 0# how to the user_ids here
  deno = 0# how to get 
  # iterate over neighbour users
  the_delui = find_delui(mu,user)
  for i in range(N):
    deno += abs(neighbour_similarities[i])
    real_neighbour_id = find_key(neighbour_index[i],dict_users)
    rating_neigh = pivot_table_nn[item][real_neighbour_id]
    delui_neigh = find_delui(mu,real_neighbour_id)
    num += ((neighbour_similarities[i]) *(rating_neigh-delui_neigh))
  if deno>0:
    term2 = num/deno
  else:
    term2 = 0
  pred_ui = the_delui + term2
  mae.append(abs(pred_ui-given_rating))
  pred.append(pred_ui)
  actuals.append(given_rating)

# print("MAE:",np.array(mae).mean())
# print("Prediction:",np.array(pred))
# print("Actual:",np.array(actuals))

normalized = [((x-min(pred)) / (max(pred) - min(pred)) * 5)+3 for x in pred]
print(np.mean(abs(np.array(normalized)-np.array(actuals))))

